# Simulation of inventory models

## Simulation procedure

### Parameter setting

In [8]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from DataSCM import period_dist_finder

In [33]:
dist_unit = norm(62000/52, 20000/52)
k=1.65
pur_cost=50
order_cost=5000
hold_cost=12 
short_cost=30
lead_time=3
time_unit= 'week'
N_iter= 1000

In [34]:
demand_vector = np.floor(dist_unit.rvs(N_iter))
demand_vector[0] = 0
demand_vector

array([   0., 1112.,  881., 1264.,  543., 1217., 1367., 1366.,  984.,
       1708., 1153.,  972., 1661., 1561.,  649.,  934., 2224.,  775.,
        607., 1102.,  985., 1829., 1938., 1333., 2002., 1485.,  761.,
        939., 1239., 1098.,  958., 1203., 1481., 1060., 1769., 1574.,
       1358.,  671., 1107., 1093., 1492., 1875., 1085., 1528., 1048.,
       1296., 1223., 1143., 1551., 1391., 1419., 1315., 1195., 1354.,
       1646., 1073., 1442., 1560.,  826., 1586., 1225., 1034.,  965.,
       1426.,  804., 1409., 1118.,  939., 1238.,  816., 1394., 1594.,
        495., 1392.,  915., 2229., 1001., 1087., 1792.,  744., 1246.,
        350.,  670., 1611., 1565., 2122., 1965., 1832., 1424., 1498.,
       1436., 1416.,  958., 1111.,  871., 1766., 1464., 1003.,  585.,
        804., 2224., 1114., 1809., 1579.,  749., 1530., 1101., 1299.,
       1377.,  891., 1037., 2173.,  387., 1074.,  944., 1087.,  973.,
       1455., 1282., 1236., 2226., 1696., 2146., 1883., 1486., 1120.,
       1240., 1501.,

### Decision Variables

In [35]:
mean_demand = dist_unit.mean()

if time_unit == 'week':
    year_demand = mean_demand * 52
    num_unit = 52
elif time_unit== 'day':
    year_demand = mean_demand * 365
    num_unit = 365
elif time_unit== 'month':
    year_demand = mean_demand * 12
    num_unit = 12

In [36]:
Q_star = np.ceil(np.sqrt((2 * order_cost * year_demand) / hold_cost))
R = Q_star / year_demand
R_unit_based = int(R*num_unit)+1

lt_dist = period_dist_finder(lead_time, 'norm', dist_unit)
small_s = np.ceil(lt_dist.mean() + k * lt_dist.std())
ltR_dist = period_dist_finder(lead_time+R_unit_based, 'norm', dist_unit)
big_s = np.ceil(ltR_dist.mean() + k * ltR_dist.std())

### main loop of simulation

In [37]:
sim_data_sq = pd.DataFrame(columns=['demand', 'order_size', 'receive', 'IOH', 'IP', 'short', 'CSL'])
sim_data_sq['demand'] = demand_vector
sim_data_sq

,demand,order_size,receive,IOH,IP,short,CSL
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1112.0,NaN,NaN,NaN,NaN,NaN,NaN
2,881.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1264.0,NaN,NaN,NaN,NaN,NaN,NaN
4,543.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
995,1272.0,NaN,NaN,NaN,NaN,NaN,NaN
996,1214.0,NaN,NaN,NaN,NaN,NaN,NaN
997,454.0,NaN,NaN,NaN,NaN,NaN,NaN
998,1025.0,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
order_vec = np.zeros(N_iter)
rec_vec = np.zeros(N_iter)
IOH_vec = np.zeros(N_iter)
IP_vec = np.zeros(N_iter)
short_vec = np.zeros(N_iter)
CSL_vec = np.zeros(N_iter)

In [39]:
order_p = -100
for n in range(N_iter):
    if n == 0:
        IOH_vec[n] = Q_star
        short_vec[n] = 0
    else:
        temp_inv = IOH_vec[n - 1] + rec_vec[n] - demand_vector[n]
        if temp_inv >= 0:
            IOH_vec[n] = temp_inv
            short_vec[n] = 0
        else:
            IOH_vec[n] = 0
            short_vec[n] = -temp_inv

    if n < order_p + lead_time:
        IP_vec[n] = IOH_vec[n] + Q_star
    else: 
        IP_vec[n] = IOH_vec[n]

    if IP_vec[n] <= small_s:
        order_vec[n] = Q_star
        if n+lead_time < N_iter:
            rec_vec[n+lead_time] = Q_star
        order_p = n
    else:
        order_vec[n] = 0

    CSL_vec[n] = short_vec[n] == 0

In [40]:
sim_data_sq.order_size = order_vec
sim_data_sq.IOH = IOH_vec
sim_data_sq.IP = IP_vec
sim_data_sq.short = short_vec
sim_data_sq.CSL = CSL_vec
sim_data_sq.receive = rec_vec
sim_data_sq

,demand,order_size,receive,IOH,IP,short,CSL
0,0.0,0.0,0.0,7188.0,7188.0,0.0,1.0
1,1112.0,0.0,0.0,6076.0,6076.0,0.0,1.0
2,881.0,0.0,0.0,5195.0,5195.0,0.0,1.0
3,1264.0,7188.0,0.0,3931.0,3931.0,0.0,1.0
4,543.0,0.0,0.0,3388.0,10576.0,0.0,1.0
...,...,...,...,...,...,...,...
995,1272.0,0.0,0.0,4815.0,4815.0,0.0,1.0
996,1214.0,7188.0,0.0,3601.0,3601.0,0.0,1.0
997,454.0,0.0,0.0,3147.0,10335.0,0.0,1.0
998,1025.0,0.0,0.0,2122.0,9310.0,0.0,1.0


### Results

In [46]:
result_data = pd.DataFrame(columns=['Policy', 'TC', 'IFR', 'CSL',
                                    'PC', 'OC', 'HC','SC', 
                                    'Q*', 'R*', 's*', 'S*'])

In [77]:
result_data.loc[0, 'Policy'] = 's,Q'
result_data.loc[0, 'TC'] = np.round(pur_cost*np.sum(order_vec) +
                                    order_cost * np.sum(order_vec > 0) +
                                    hold_cost * np.sum(IOH_vec)/num_unit +
                                    short_cost * np.sum(short_vec),
                                   1)

result_data.loc[0, 'PC'] = pur_cost*np.sum(order_vec)
result_data.loc[0, 'OC'] = order_cost*np.sum(order_vec > 0)
result_data.loc[0, 'HC'] = np.round(hold_cost*np.sum(IOH_vec)/num_unit,  1)
result_data.loc[0, 'SC'] = short_cost*np.sum(short_vec)

result_data.loc[0, 'IFR'] = np.round(1-np.sum(short_vec)/np.sum(order_vec),  4)
result_data.loc[0, 'CSL'] = np.mean(CSL_vec)
result_data.loc[0, 'Q*'] = Q_star
result_data.loc[0, 'R*'] = '-'
result_data.loc[0, 's*'] = small_s
result_data.loc[0, 'S*'] = '-'

In [78]:
result_data

,Policy,TC,IFR,CSL,PC,OC,HC,SC,Q*,R*,s*,S*
0,"s,Q",62718984.6,0.9981,0.992,60738600.0,845000,1064824.6,70560.0,7188.0,-,4677.0,-
